<a href="https://colab.research.google.com/github/xamevou/someJupyterNotebooks/blob/main/KeyAndAddressGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keys and addresses in Bitcoin

Taken from example 5 of chaper 4 in Antonopoulos' book.

First, we need the bitcoin package.

In [1]:
# !pip install bitcoin   # Uncomment if needed
from __future__ import print_function
import bitcoin

## Generate a random private key

We generate a random private key, but it must be valid. In other words, it must be an integer between 1 and `bitcoin.N`. This number, in the standard secp256k1, is the size $p$ of the Finite Field $F_p$ where the elliptic curve $y^2=x^3+7$ is defined. Its value is: $2^{256} - 2^{32} - 2^9 - 2^8 - 2^7 - 2^6 - 2^4 - 1$.

*Note: In the book, example 4.3 is in cpp. If you want to reproduce that example here, change `bitcoin.random_key()` with the following string: `private_key = "038109007313a5807b2eccc082c8c3fbb988a973cacf1a7df9ce725c31b14776"`*

In [2]:
valid_private_key = False
while not valid_private_key:    
    private_key = bitcoin.random_key()
    decoded_private_key = bitcoin.decode_privkey(private_key, 'hex')
    valid_private_key = 0 < decoded_private_key < bitcoin.N

print("Private Key (hex) is: ", private_key)
print("Private Key (decimal) is: ", decoded_private_key)

Private Key (hex) is:  f0a3b14b77f86afca087fdfdd68fa5917c7563648db44b76291d3cf031d0c643
Private Key (decimal) is:  108844303375276547733718999297423996970605593588872697801111435468685966362179


### WIF
The private key is now converted to Wallet Import Format (WIF). Supposedly, the WIF format encoding allows easy copy of a big number.

In [3]:
wif_encoded_private_key = bitcoin.encode_privkey(decoded_private_key, 'wif')
print("Private Key (WIF) is: ", wif_encoded_private_key)

Private Key (WIF) is:  5KeGPj3PuPQZ7AmphBqCTJPro6ycCebfzwqZ5Vm2N6s6h3s3MEV


We add suffix "01" to create a "compressed" private key.

In [4]:
compressed_private_key = private_key + '01'
print("Private Key Compressed (hex) is: ", compressed_private_key)

Private Key Compressed (hex) is:  f0a3b14b77f86afca087fdfdd68fa5917c7563648db44b76291d3cf031d0c64301


Now we generate a WIF format from the compressed private key (WIF-compressed)

In [5]:
wif_compressed_private_key = bitcoin.encode_privkey(bitcoin.decode_privkey(compressed_private_key, 'hex'), 'wif')
print("Private Key (WIF-Compressed) is: ", wif_compressed_private_key)

Private Key (WIF-Compressed) is:  L5HUwJZa6DtxiE8qWNojZ5mtnVNrNQx38m3JaqwK28t87Nj3SkJV


# From Private Key to Public Key

We multiply the EC generator point G, `bitcoin.G`, with the private key, to get a public key point. The G point, if you are interested, is $(55066263022277343669578718895168534326250603453777594175500187360389116729240, 32670510020758816978083085130507043184471273380659243275938904335757337482424)$

In [6]:
public_key = bitcoin.fast_multiply(bitcoin.G, decoded_private_key)
print("Public Key (x,y) coordinates is:", public_key)

Public Key (x,y) coordinates is: (94975686041702978464708086685935502719370584860157694448638800434455447892734, 18152670299822949699444414757492961746715405978332902079947279307753095118501)


Now we encode that public key as hex, prefix 04.

In [7]:
hex_encoded_public_key = bitcoin.encode_pubkey(public_key, 'hex')
print("Public Key (hex) is:", hex_encoded_public_key)

Public Key (hex) is: 04d1fa5554510101d988a24af7910177a7a9eaa881f0d493d8f0fa92d1f82c72fe28220c1b466920c95eff790e90ed9d6106f52c6bb18b1e241f6bca3c7e8b3ea5


We compress the public key, adjusting prefix depending on whether $y$ is even or odd

### Note on "compression"
The public key is a point. Here, "compression" refers to taking only the x component of the (x,y) pair that is the public key. Every use of "compression" points to that.

In [8]:
(public_key_x, public_key_y) = public_key
compressed_prefix = '02' if (public_key_y % 2) == 0 else '03'
hex_compressed_public_key = compressed_prefix + (bitcoin.encode(public_key_x, 16).zfill(64))
print("Compressed Public Key (hex) is:", hex_compressed_public_key)

Compressed Public Key (hex) is: 03d1fa5554510101d988a24af7910177a7a9eaa881f0d493d8f0fa92d1f82c72fe


# Generation of the bitcoin address from the public key

At last, we generate the address that corresponds to the public key.

In [9]:
print("Bitcoin Address (b58check) is:", bitcoin.pubkey_to_address(public_key))

Bitcoin Address (b58check) is: 13KxgxSRfiJ8evByUPpUSVH8Rhe8kvmL8c


The address is now compressed.

In [10]:
# Generate compressed bitcoin address from compressed public key
print("Compressed Bitcoin Address (b58check) is:",
      bitcoin.pubkey_to_address(hex_compressed_public_key))

Compressed Bitcoin Address (b58check) is: 1JmSys1f8WKWDXwakcrnRACvU3Ps97NNFX


And that's all !

## NOTE
As you see, this process proceeds in a one-to-one fashion: one seed, one private key, one public key, one address. Obviously we want more flexibility in our wallets. That's what the seed words are for: with the words, several seeds are generated, which lead to several private keys, etc. 

From the words to the keys, that should be a topic for another Notebook. 